https://pythonviz.com/pandas/3-ways-to-use-pandas-apply-in-python/
甚麼是 pandas apply？
簡單來說，pandas 的 apply 是一個在 pandas dataframe 加入新列（Column）的指令。這個指令在整合（Transform）數據時基本上時無可避免，例如我們需要加入新的列，是相加 2 個列的結果等。

除了整合數據以外，我其中一個比較喜歡的用法是搭配 pandas groupby 一起使用，能夠簡單明瞭地把數據分類。

In [1]:
import pandas as pd

df = pd.DataFrame(
    {
      'Name':   ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O'],
      'Sex':    ['M', 'M', 'M', 'F', 'F', 'M', 'F', 'M', 'F', 'M', 'M', 'M', 'F', 'M', 'M'],
      'Age':    [38, 28, 31, 34, 28, 28, 36, 33, 22, 39, 22, 24, 31, 29, 22],
      'Height': [1.74, 1.51, 1.67, 1.87, 1.8, 1.51, 1.85, 1.89, 1.81, 1.72, 1.75, 1.64, 1.9, 1.62, 1.61],
      'Weight': [45, 63, 39, 45, 67, 66, 53, 45, 72, 46, 58, 44, 73, 70, 51],
    }
)
print(df)

   Name Sex  Age  Height  Weight
0     A   M   38    1.74      45
1     B   M   28    1.51      63
2     C   M   31    1.67      39
3     D   F   34    1.87      45
4     E   F   28    1.80      67
5     F   M   28    1.51      66
6     G   F   36    1.85      53
7     H   M   33    1.89      45
8     I   F   22    1.81      72
9     J   M   39    1.72      46
10    K   M   22    1.75      58
11    L   M   24    1.64      44
12    M   F   31    1.90      73
13    N   M   29    1.62      70
14    O   M   22    1.61      51


### apply 的兩種型態（lambda 與 non-lambda）


In [2]:
df_eg1 = df.copy()

# 第 1 個用法
def BMI_1(r):
  return round(r['Weight'] / (r['Height'] ** 2), ndigits=2) #留意我們的輸入函數（input）是 r，代表 pandas dataframe 裡面其中一行（row）。

df_eg1['BMI_apply1'] = df_eg1.apply(BMI_1, axis=1)  #We want to apply the function written above on the dataframe df_eg1
#BMI_1 輸出一個 float 數字，是透過融合 r['Weight'] 和 r['Height'] 而成。我們能夠像存取列表（List）一樣，以 r['Column Name'] 獲得每一行（Row）的某一列（Column）的值。

# 第 2 個用法
# BMI_2 這個功能與 BMI_1 略有不同。分別在於 BMI_2 直接輸入 2 個 float 數字，而不是 pandas dataframe 的行。
def BMI_2(weight, height):
  return round(weight / (height ** 2), ndigits=2)

df_eg1['BMI_apply2'] = df_eg1.apply(lambda r: BMI_2(r['Weight'],r['Height']), axis=1)

print(df_eg1)
#apply 裡的函數 axis=1 代表我們以行（Row）作單位，而 0 代表我們以列（Column）作單位。有 99% 的情況我們都以行（Row）為單位！

   Name Sex  Age  Height  Weight  BMI_apply1  BMI_apply2
0     A   M   38    1.74      45       14.86       14.86
1     B   M   28    1.51      63       27.63       27.63
2     C   M   31    1.67      39       13.98       13.98
3     D   F   34    1.87      45       12.87       12.87
4     E   F   28    1.80      67       20.68       20.68
5     F   M   28    1.51      66       28.95       28.95
6     G   F   36    1.85      53       15.49       15.49
7     H   M   33    1.89      45       12.60       12.60
8     I   F   22    1.81      72       21.98       21.98
9     J   M   39    1.72      46       15.55       15.55
10    K   M   22    1.75      58       18.94       18.94
11    L   M   24    1.64      44       16.36       16.36
12    M   F   31    1.90      73       20.22       20.22
13    N   M   29    1.62      70       26.67       26.67
14    O   M   22    1.61      51       19.68       19.68


df.apply(BMI_1, axis=1)
- 這裡 apply 的第一個函數（input）是一個 Python 的功能。BMI_1 是一個 Python 的功能（準確來說是一個功能的 Object），所以我們直接把它作為 apply 的主函數。
- 自訂的 Python 功能（例如 BMI_1）需要處理 pandas dataframe 行（row）的輸入函數。

df.apply(lambda r: BMI_2(r['Weight'], r['Height']), axis=1)
- 個人比較喜歡這個用法。我們加入 lambda r 的語法，直接在 apply，而不是 BMI_2 指定哪個列（column）是 Weight 和 Height。

- 這個方法的 Python 功能（BMI_2）只需要處理 weight 和 height 值作為功能輸入函數。好處在於 BMI_2 可以獨立在其他位置使用，而不像 BMI_1 般只能用於 pandas dataframe。

#### 進階：pandas apply 同時輸出多個列（Column）
除了以上的寫法外，apply 還能夠同時增加多於 1 個列。這個用於有時處理數據會同時輸出 2 個或更多的列（Column），而這些列是互相關聯（dependent），不能獨自計算。



In [3]:
import math
import random

df_eg2 = df.copy()

def many_columns(x):
  # 第 1 個列, adding another column
  x['Age_group'] = math.floor(x['Age'] / 10) * 10
  
  # 第 2 個列
  if x['Height'] > 1.7:
    x['Height_group'] = 'Tall'
  else:
    x['Height_group'] = 'Short'

  # 第 3 個列
  x['Random_group'] = random.randint(1,3)
  
  # 回傳結果
  return x

df_eg2 = df_eg2.apply(many_columns, axis=1)
print(df_eg2)

   Name Sex  Age  Height  Weight  Age_group Height_group  Random_group
0     A   M   38    1.74      45         30         Tall             1
1     B   M   28    1.51      63         20        Short             2
2     C   M   31    1.67      39         30        Short             2
3     D   F   34    1.87      45         30         Tall             1
4     E   F   28    1.80      67         20         Tall             1
5     F   M   28    1.51      66         20        Short             3
6     G   F   36    1.85      53         30         Tall             2
7     H   M   33    1.89      45         30         Tall             3
8     I   F   22    1.81      72         20         Tall             1
9     J   M   39    1.72      46         30         Tall             3
10    K   M   22    1.75      58         20         Tall             3
11    L   M   24    1.64      44         20        Short             3
12    M   F   31    1.90      73         30         Tall             1
13    

與簡單例子不同之處在於上面我們把 apply 的結果輸出成 df['new column'] ，但這裡我們直接把 apply 的結果輸出成 df。前者是 dataframe 的一列（Column），而後者是整個 dataframe。



#### 進階：以 apply 製做自訂數據類型（object）


In [10]:
class Person:
  def __init__(self, sex, height, weight, age):
    self.sex = sex
    self.height = height
    self.weight = weight
    self.age = age
  
  def describe(self):
    if self.sex == 'M':
      sex_describe = '男生'
    else:
      sex_describe = '女生'
    
    if self.height > 1.7:
      height_describe = '高高'
    else:
      height_describe = '矮矮'

    if self.weight > 60:
      weight_describe = '胖胖'
    else:
      weight_describe = '瘦瘦'

    if self.age > 30:
      age_describe = '青年'
    else:
      age_describe = '少年'

    return '我是一個' + height_describe + weight_describe + '的' + age_describe + sex_describe
#留意我們先定義了何謂一個「人」（Person）的數據模型，並賦予這個數據模型一個功能去描述這個「人」的特徵。



In [11]:
df_eg3 = df.copy()

df_eg3['Object'] = df_eg3.apply(lambda x: Person(x['Sex'],x['Height'],x['Weight'],x['Age']), axis=1) # We are storing the info of each row as an 
#object called person, then save it to the dataframe
df_eg3['Object_describe'] = df_eg3.apply(lambda x: x['Object'].describe(), axis=1) # Then we add another column describing each object's details

print(df_eg3)

   Name Sex  Age  Height  Weight                                   Object  \
0     A   M   38    1.74      45  <__main__.Person object at 0x130f93ee0>   
1     B   M   28    1.51      63  <__main__.Person object at 0x131073e80>   
2     C   M   31    1.67      39  <__main__.Person object at 0x131073b50>   
3     D   F   34    1.87      45  <__main__.Person object at 0x130dcd850>   
4     E   F   28    1.80      67  <__main__.Person object at 0x130dcd1f0>   
5     F   M   28    1.51      66  <__main__.Person object at 0x130dcdf70>   
6     G   F   36    1.85      53  <__main__.Person object at 0x130dcd340>   
7     H   M   33    1.89      45  <__main__.Person object at 0x130dcd760>   
8     I   F   22    1.81      72  <__main__.Person object at 0x130dcd7f0>   
9     J   M   39    1.72      46  <__main__.Person object at 0x130dcd6d0>   
10    K   M   22    1.75      58  <__main__.Person object at 0x130dcdd90>   
11    L   M   24    1.64      44  <__main__.Person object at 0x130ed8220>   

In [13]:
Lee = Person('M', 1.73, 68, 34)
Lee.describe()

'我是一個高高胖胖的青年男生'